In [1]:
!pip install wget
!apt-get -y install sox libsndfile1 ffmpeg
!pip install unidecode
BRANCH = 'r1.4.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml
!pip install matplotlib==3.1.3
!pip install torchtext==0.10.1
!pip install torchaudio==0.9.1

In [2]:
data_dir = '/kaggle/working'

In [3]:
import os
import tarfile
import wget

def load(golos_url, fname):
    if not os.path.exists(os.path.join(data_dir, fname)):
        golos_path = wget.download(golos_url, os.path.join(data_dir, fname))
        print(f"Dataset downloaded at: {golos_path}")
    else:
        print("Tarfile already exists.")
        golos_path = os.path.join(data_dir, fname)
    return golos_path

golos_path = load("https://sc.link/Kqr", "test.tar")

if not os.path.exists(os.path.join(data_dir, '/test/')):
    tar = tarfile.open(golos_path)
    tar.extractall(path=data_dir)

Tarfile already exists.


In [ ]:
import nemo
import nemo.collections.asr as nemo_asr

load("https://sc.link/ZMv", "QuartzNet15x5_golos.nemo")
asr_model = nemo_asr.models.EncDecCTCModel.restore_from(os.path.join(data_dir, "QuartzNet15x5_golos.nemo"))

In [21]:
# --- Building Manifest Files --- #
import json

# Function to build a manifest
def build_manifest(manifest_rel, manifest_abs):
    manifest_path = os.path.split(os.path.abspath(manifest_rel))[0]
    with open(manifest_rel, 'r') as fin:
        with open(manifest_abs, 'w') as fout:
            for line in fin:
                metadata = json.loads(line)
                metadata["audio_filepath"]=os.path.join(manifest_path, metadata["audio_filepath"])
                json.dump(metadata, fout)
                fout.write('\n')
                
# Building Manifests
print("******")
train_rel = os.path.join(data_dir, 'test/farfield/manifest.jsonl')
train_abs = os.path.join(data_dir, 'test/farfield/farfield.jsonl')
if not os.path.isfile(train_abs):
    build_manifest(train_rel, train_abs)
test_manifest = train_abs
print("test_manifest", test_manifest)

train_rel = os.path.join(data_dir, 'test/crowd/manifest.jsonl')
train_abs = os.path.join(data_dir, 'test/crowd/crowd.jsonl')
if not os.path.isfile(train_abs):
    build_manifest(train_rel, train_abs)
train_manifest = train_abs
print("train_manifest", train_manifest)

******
test_manifest /kaggle/working/test/farfield/farfield.jsonl
train_manifest /kaggle/working/test/crowd/crowd.jsonl


In [22]:
# --- Config Information ---#
try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel_yaml import YAML
config_path = './configs/config.yaml'

yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)

In [23]:
params["labels"] = asr_model.decoder.vocabulary
params["model"]["train_ds"]["labels"] = asr_model.decoder.vocabulary
params["model"]["validation_ds"]["labels"] = asr_model.decoder.vocabulary
params["model"]["decoder"]["vocabulary"] = asr_model.decoder.vocabulary
params["model"]["decoder"]["num_classes"] = len(asr_model.decoder.vocabulary)

In [24]:
params = asr_model._cfg
params['train_ds']['parser'] = 'base'
params['validation_ds']['parser'] = 'base'
params['test_ds']['parser'] = 'base'
params['train_ds']['manifest_filepath'] = train_manifest
params['train_ds']['batch_size'] = 32
params['validation_ds']['manifest_filepath'] = test_manifest
params['validation_ds']['batch_size'] = 32
params['test_ds']['manifest_filepath'] = test_manifest
params['test_ds']['batch_size'] = 32

In [25]:
import copy
from omegaconf import DictConfig

new_opt = copy.deepcopy(params['optim'])
new_opt['lr'] = 0.0001
asr_model.setup_optimization(optim_config=DictConfig(new_opt))

[NeMo W 2021-11-30 19:49:12 modelPT:436] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-11-30 19:49:12 modelPT:544] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0001
        weight_decay: 0.001
    )


[NeMo W 2021-11-30 19:49:12 lr_scheduler:605] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


(Novograd (
 Parameter Group 0
     amsgrad: False
     betas: [0.9, 0.98]
     eps: 1e-08
     grad_averaging: False
     lr: 0.0001
     weight_decay: 0.001
 ),
 None)

In [26]:
asr_model.setup_training_data(train_data_config=params['train_ds'])
asr_model.setup_test_data(test_data_config=params['test_ds'])
asr_model.setup_validation_data(val_data_config=params['validation_ds'])

[NeMo I 2021-11-30 19:49:17 collections:173] Dataset loaded with 9990 files totalling 11.17 hours
[NeMo I 2021-11-30 19:49:17 collections:174] 4 files were filtered totalling 0.03 hours
[NeMo I 2021-11-30 19:49:17 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 19:49:17 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-30 19:49:17 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 19:49:17 collections:174] 0 files were filtered totalling 0.00 hours


In [28]:
asr_model.cuda()

wer_nums = []
wer_denoms = []

for test_batch in asr_model.test_dataloader():
    test_batch = [x.cuda() for x in test_batch]
    targets = test_batch[2]
    targets_lengths = test_batch[3]        
    log_probs, encoded_len, greedy_predictions = asr_model(
        input_signal=test_batch[0], input_signal_length=test_batch[1]
    )
    asr_model._wer.update(greedy_predictions, targets, targets_lengths)
    _, wer_num, wer_denom = asr_model._wer.compute()
    asr_model._wer.reset()
    wer_nums.append(wer_num.detach().cpu().numpy())
    wer_denoms.append(wer_denom.detach().cpu().numpy())

    del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

WER = 0.6002147458840372


In [18]:
import copy
from omegaconf import DictConfig
import pytorch_lightning as pl
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint


trainer = pl.Trainer(gpus=1, max_epochs=5, val_check_interval=0.2,
                     precision=16, callbacks=[ModelCheckpoint(dirpath=data_dir)])   
trainer.fit(asr_model)

[NeMo W 2021-11-30 19:10:03 nemo_logging:349] /opt/conda/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory /kaggle/working exists and is not empty.
      rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
    
[NeMo W 2021-11-30 19:10:03 modelPT:436] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-11-30 19:10:03 modelPT:544] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0001
        weight_decay: 0.001
    )


[NeMo W 2021-11-30 19:10:03 lr_scheduler:605] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [19]:
trainer.test()

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 6.556389331817627, 'test_wer': 0.16404199600219727}
--------------------------------------------------------------------------------


[{'test_loss': 6.556389331817627, 'test_wer': 0.16404199600219727}]